# Python Scraping

- 도서관 정보나루 
- 웹 스크래핑을 통해 원하는 도서관의 대출 데이터를 다운로드 하기

- https://dsc-sookmyung.tistory.com/85


In [1]:
import requests
from bs4 import BeautifulSoup


# 로그 메시지 숨기기
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [2]:
url = "https://www.data4library.kr/openDataL"
response = requests.get(url, verify = False) # SSLError

In [3]:
# SSLError 
# https://blog.naver.com/PostView.nhn?blogId=dsz08082&logNo=222093530720&parentCategoryNo=&categoryNo=55&viewDate=&isShowPopularPosts=true&from=search

In [4]:
html = response.text

In [5]:
soup = BeautifulSoup(html, "html5lib")

In [6]:
# CSS 셀렉터 : id 셀렉터로 검색 칸 select
tags = soup.select("#srchText") 
tags

[<input id="srchText" name="srchText" placeholder="Search" title="검색" type="text" value=""/>]

In [7]:
tag= tags[0]
tag.text

''

In [8]:
# 다중 셀렉터로 text select 
tags = soup.select(".link_td a ") 
# text에서 도서관명만 추출
lib_name = tags[0].text.split(' ')[0] 

# 특수문자 제거
# lib_name = ''.join(filter(str.isalnum, tags[0].text))

# Python Selenium 

- https://sites.google.com/a/chromium.org/chromedriver/downloads 

1. 

1-1.
- 도서관 목록 엑셀 파일 불러오기
- 도서관명을 리스트로 저장
- 리스트의 모든 도서관명을 검색  

1-2.
- https://www.data4library.kr/openDataL
- 검색 입력 칸 클릭 > 검색 도서관명 입력 > 검색 버튼 클릭
- 검색 결과 첫번째 도서관 클릭 > 첫번째 엑셀버튼 클릭


2.
- https://www.data4library.kr/libDataL 참여도서관 목록 
- ~~검색 입력 칸 클릭 > 검색 도서관명 입력 > 검색 버튼 클릭~~ 
- ~~검색 결과 첫번째 도서관 클릭 > 도서관 코드 크롤링~~
- ~~도서관명 + 도서관 코드 저장~~

## 1-1

In [2]:
import pandas as pd
import openpyxl
import xlsxwriter

df = pd.read_excel('총람_도서관_목록(정보나루)_1101.xlsx', engine = 'openpyxl', sheet_name = 1)
df

,주소1,도서관명,정보나루_도서관명,주소,정보나루_주소,비도서(점),연간증가 비도서(점),건물면적,규모
0,대전광역시,대전광역시한밭도서관,한밭도서관,대전광역시 중구 서문로 10 문화동한밭도서관,대전광역시 중구 서문로 10,62628,587,22485.39,5
1,광주광역시,광주학생교육문화회관,광주학생교육문화회관,광주광역시 서구 상무민주로 61 쌍촌동61 광주학생교육문화회관,광주광역시 서구 상무민주로 61,4312,251,18350.13,5
2,울산광역시,울산도서관,울산도서관,울산광역시 남구 꽃대나리로 140 여천동울산도서관,울산광역시 남구 꽃대나리로 140,4072,393,15176.65,5
3,경기도,경기평생교육학습관,경기평생교육학습관,경기도 수원시 권선구 권중로 55 (권선동)경기평생교육학습관,경기도 수원시 권선구 권중로 55,7325,640,14900.00,5
4,광주광역시,광주학생독립운동기념회관,광주학생독립운동기념회관,광주광역시 서구 학생독립로30 광주학생독립운동기념회관,광주광역시 서구 학생독립로 30,1758,203,13783.20,5
...,...,...,...,...,...,...,...,...,...
696,서울특별시,방배도서관,방배도서관,서울특별시 서초구 방배로 40 (방배동)방배3동 주민센터(신청사) 4층,서울특별시 서초구 방배로 40,0,0,271.00,1
697,경상남도,상북어린이도서관,양산시립 상북어린이도서관,경상남도 양산시 상북면 반회서7길 14-16상북어린이도서관,경상남도 양산시 상북면 반회서7길 14-16,274,68,265.34,1
698,서울특별시,돌마리도서관,돌마리도서관,서울특별시 송파구 백제고분로37길 16 석촌동석촌동주민센터 5층,서울특별시 송파구 백제고분로37길 16,0,0,265.00,1
699,경기도,용인시구갈희망누리도서관,구갈희망누리도서관,경기도 용인시 기흥구 갈곡로12번길 1 (구갈동)근린공원 내,경기도 용인시 기흥구 갈곡로12번길 1,0,0,264.00,1


In [3]:
df['규모'].unique()

array([5, 4, 3, 2, 1], dtype=int64)

In [5]:
# 규모별 건물 면적 순으로 상위 3개 행 선택하기 
df_sort_group_top3 = df.sort_values(by="건물면적",ascending=False).groupby("규모").head(3)
df_sort_group_top3.head(50)

,주소1,도서관명,정보나루_도서관명,주소,정보나루_주소,비도서(점),연간증가 비도서(점),건물면적,규모
0,대전광역시,대전광역시한밭도서관,한밭도서관,대전광역시 중구 서문로 10 문화동한밭도서관,대전광역시 중구 서문로 10,62628,587,22485.39,5
1,광주광역시,광주학생교육문화회관,광주학생교육문화회관,광주광역시 서구 상무민주로 61 쌍촌동61 광주학생교육문화회관,광주광역시 서구 상무민주로 61,4312,251,18350.13,5
2,울산광역시,울산도서관,울산도서관,울산광역시 남구 꽃대나리로 140 여천동울산도서관,울산광역시 남구 꽃대나리로 140,4072,393,15176.65,5
101,경기도,남양주시오남도서관,남양주시 오남도서관,경기도 남양주시 오남읍 양지로240번길 12오남도서관,경기도 남양주시 오남읍 양지로240번길 12,2877,290,5454.00,4
102,서울특별시,서울특별시교육청고척도서관,고척도서관,서울특별시 구로구 고척로45길 31 (고척동)고척로45길 31 (고척동),서울특별시 구로구 고척로45길 31,11423,595,5437.66,4
103,경기도,부천시립상동도서관,부천시립상동도서관,경기도 부천시 상이로 12 (상동)상이로 12(상동도서관),경기도 부천시 상이로 12,4723,41,5426.00,4
216,대구광역시,대구광역시립두류도서관,대구광역시립두류도서관,대구광역시 달서구 공원순환로 8 두류동8 두류도서관,대구광역시 달서구 공원순환로 8,6145,140,3461.00,3
217,경상남도,김해지혜의바다,경상남도교육청 김해지혜의바다도서관,경상남도 김해시 주촌면 서부로1541번길 8김해지혜의바다,경상남도 김해시 주촌면 서부로1541번길 8,0,0,3449.19,3
218,충청남도,충청남도교육청학생교육문화원,충청남도교육청 학생교육문화원,충청남도 천안시 동남구 옛농고1길 41 원성동충청남도교육청학생교육문화원,충청남도 천안시 동남구 옛농고1길 41,3419,305,3421.00,3
306,경기도,가평군립청평도서관,가평군 청평도서관,경기도 가평군 청평면 은고개로 3131,경기도 가평군 청평면 은고개로 31,1545,17,2485.23,2


In [31]:
lib_list = df_sort_group_top3['정보나루_도서관명']
lib_list= lib_list.tolist()
lib_list

['한밭도서관',
 '광주학생교육문화회관',
 '울산도서관',
 '남양주시 오남도서관',
 '고척도서관',
 '부천시립상동도서관',
 '대구광역시립두류도서관',
 '경상남도교육청 김해지혜의바다도서관',
 '충청남도교육청 학생교육문화원',
 '가평군 청평도서관',
 '청주흥덕도서관',
 '대구 동구 안심도서관',
 '시흥시 군자도서관',
 '도봉아이나라도서관',
 '경기평택교육도서관']

In [32]:
lib_list[5] = '남양주시 진접푸른숲도서관'

In [33]:
lib_list

['한밭도서관',
 '광주학생교육문화회관',
 '울산도서관',
 '남양주시 오남도서관',
 '고척도서관',
 '남양주시 진접푸른숲도서관',
 '대구광역시립두류도서관',
 '경상남도교육청 김해지혜의바다도서관',
 '충청남도교육청 학생교육문화원',
 '가평군 청평도서관',
 '청주흥덕도서관',
 '대구 동구 안심도서관',
 '시흥시 군자도서관',
 '도봉아이나라도서관',
 '경기평택교육도서관']

In [12]:
len(lib_list)

15

## 1-2

In [18]:
import selenium
from selenium import webdriver

options = webdriver.ChromeOptions()
options.binary_location = 'C:/Program Files/Google/Chrome/Application/chrome.exe'

browser = webdriver.Chrome('chromedriver.exe', options = options)

In [20]:
URL = "https://www.data4library.kr/openDataL"

driver = webdriver.Chrome(executable_path='chromedriver')
driver.get(url=URL)

In [21]:
from selenium.webdriver.common.keys import Keys

In [22]:
import time 

def download_file(lib_name):
    # 검색 창에 도서관명 입력
    search_box = driver.find_element_by_xpath('//*[@id="srchText"]')
    search_box.send_keys(lib_name)
    print(lib_name + " 입력")
    time.sleep(1)

    # 검색 버튼 클릭
    search_btn = driver.find_element_by_xpath('//*[@id="pagef"]/div[3]/div[2]/div/a')
    search_btn.click()
    print("클릭")
    time.sleep(1)


    # 첫번째 도서관 클릭
    lib_btn = driver.find_element_by_xpath('//*[@id="pagef"]/div[4]/table/tbody/tr/td[1]/a')
    lib_btn.click()
    print("도서관 클릭")
    time.sleep(1)

    # 첫번째 엑셀 파일 클릭
    file_btn = driver.find_element_by_xpath('//*[@id="sb-site"]/section/div[5]/div[3]/table/tbody/tr[1]/td[4]/a[2]')
    file_btn.click()
    print("엑셀파일 클릭")
    time.sleep(5)

In [23]:
URL = "https://www.data4library.kr/openDataL"

driver = webdriver.Chrome(executable_path='chromedriver')
driver.get(url=URL)

for lib in lib_list:
    download_file(lib)
    
    # 뒤로가기
    driver.back()
    print("뒤로가기")
    time.sleep(1)
    
    # 검색창 텍스트 지우기
    search_box = driver.find_element_by_xpath('//*[@id="srchText"]')
    search_box.clear()
    print("텍스트 지우기")
    time.sleep(1)

한밭도서관 입력
클릭
도서관 클릭
엑셀파일 클릭
뒤로가기
텍스트 지우기
광주학생교육문화회관 입력
클릭
도서관 클릭
엑셀파일 클릭
뒤로가기
텍스트 지우기
울산도서관 입력
클릭
도서관 클릭
엑셀파일 클릭
뒤로가기
텍스트 지우기
남양주시 오남도서관 입력
클릭
도서관 클릭
엑셀파일 클릭
뒤로가기
텍스트 지우기
고척도서관 입력
클릭
도서관 클릭
엑셀파일 클릭
뒤로가기
텍스트 지우기
부천시립상동도서관 입력
클릭
도서관 클릭
엑셀파일 클릭
뒤로가기
텍스트 지우기
대구광역시립두류도서관 입력
클릭
도서관 클릭
엑셀파일 클릭
뒤로가기
텍스트 지우기
경상남도교육청 김해지혜의바다도서관 입력
클릭
도서관 클릭
엑셀파일 클릭
뒤로가기
텍스트 지우기
충청남도교육청 학생교육문화원 입력
클릭
도서관 클릭
엑셀파일 클릭
뒤로가기
텍스트 지우기
가평군 청평도서관 입력
클릭
도서관 클릭
엑셀파일 클릭
뒤로가기
텍스트 지우기
청주흥덕도서관 입력
클릭
도서관 클릭
엑셀파일 클릭
뒤로가기
텍스트 지우기
대구 동구 안심도서관 입력
클릭
도서관 클릭
엑셀파일 클릭
뒤로가기
텍스트 지우기
시흥시 군자도서관 입력
클릭
도서관 클릭
엑셀파일 클릭
뒤로가기
텍스트 지우기
도봉아이나라도서관 입력
클릭
도서관 클릭
엑셀파일 클릭
뒤로가기
텍스트 지우기
경기평택교육도서관 입력
클릭
도서관 클릭
엑셀파일 클릭
뒤로가기
텍스트 지우기


# 다운 받은 데이터 정리

- 기간 : 2019.01 ~  2021.10 까지만
- 월별로 합계 내기 
- 한 파일에 저장 

In [13]:
import os

In [14]:
file_list = []
path = r"C:\Users\user\Desktop\도서관\스크래핑\대출목록"
# os.listdir()

for i in os.listdir(path):
    # if '대출목록' in i:
    file_list.append(i)

file_list

['가평군 청평도서관 장서 대출목록 (2021년 10월).xlsx',
 '경기평택교육도서관 장서 대출목록 (2021년 10월).xlsx',
 '경상남도교육청 김해지혜의바다도서관 장서 대출목록 (2021년 10월).xlsx',
 '고척도서관 장서 대출목록 (2021년 10월).xlsx',
 '광주학생교육문화회관 장서 대출목록 (2021년 10월).xlsx',
 '남양주시 오남도서관 장서 대출목록 (2021년 10월).xlsx',
 '남양주시 진접푸른숲도서관 장서 대출목록 (2021년 10월).xlsx',
 '대구 동구 안심도서관 장서 대출목록 (2021년 10월).xlsx',
 '대구광역시립두류도서관 장서 대출목록 (2021년 10월).xlsx',
 '도봉아이나라도서관 장서 대출목록 (2021년 10월).xlsx',
 '시흥시 군자도서관 장서 대출목록 (2021년 10월).xlsx',
 '울산도서관 장서 대출목록 (2021년 10월).xlsx',
 '청주흥덕도서관 장서 대출목록 (2021년 10월).xlsx',
 '충청남도교육청 학생교육문화원 장서 대출목록 (2021년 10월).xlsx',
 '한밭도서관 장서 대출목록 (2021년 10월).xlsx']

In [15]:
len(file_list)

15

In [16]:
for file in file_list:
    idx = file.find('장')
    lib_name = file[:idx-1]
    print(file[:file.find('장') - 1])

가평군 청평도서관
경기평택교육도서관
경상남도교육청 김해지혜의바다도서관
고척도서관
광주학생교육문화회관
남양주시 오남도서관
남양주시 진접푸른숲도서관
대구 동구 안심도서관
대구광역시립두류도서관
도봉아이나라도서관
시흥시 군자도서관
울산도서관
청주흥덕도서관
충청남도교육청 학생교육문화원
한밭도서관


In [17]:
tmp = list()

for lib_list in lib_list:
    for file in file_list:
        if lib_list in file:
            print(lib_list + " " + file)
            tmp.append(file)

한밭도서관 한밭도서관 장서 대출목록 (2021년 10월).xlsx
광주학생교육문화회관 광주학생교육문화회관 장서 대출목록 (2021년 10월).xlsx
울산도서관 울산도서관 장서 대출목록 (2021년 10월).xlsx
남양주시 오남도서관 남양주시 오남도서관 장서 대출목록 (2021년 10월).xlsx
고척도서관 고척도서관 장서 대출목록 (2021년 10월).xlsx
남양주시 진접푸른숲도서관 남양주시 진접푸른숲도서관 장서 대출목록 (2021년 10월).xlsx
대구광역시립두류도서관 대구광역시립두류도서관 장서 대출목록 (2021년 10월).xlsx
경상남도교육청 김해지혜의바다도서관 경상남도교육청 김해지혜의바다도서관 장서 대출목록 (2021년 10월).xlsx
충청남도교육청 학생교육문화원 충청남도교육청 학생교육문화원 장서 대출목록 (2021년 10월).xlsx
가평군 청평도서관 가평군 청평도서관 장서 대출목록 (2021년 10월).xlsx
청주흥덕도서관 청주흥덕도서관 장서 대출목록 (2021년 10월).xlsx
대구 동구 안심도서관 대구 동구 안심도서관 장서 대출목록 (2021년 10월).xlsx
시흥시 군자도서관 시흥시 군자도서관 장서 대출목록 (2021년 10월).xlsx
도봉아이나라도서관 도봉아이나라도서관 장서 대출목록 (2021년 10월).xlsx
경기평택교육도서관 경기평택교육도서관 장서 대출목록 (2021년 10월).xlsx


In [18]:
tmp

['한밭도서관 장서 대출목록 (2021년 10월).xlsx',
 '광주학생교육문화회관 장서 대출목록 (2021년 10월).xlsx',
 '울산도서관 장서 대출목록 (2021년 10월).xlsx',
 '남양주시 오남도서관 장서 대출목록 (2021년 10월).xlsx',
 '고척도서관 장서 대출목록 (2021년 10월).xlsx',
 '남양주시 진접푸른숲도서관 장서 대출목록 (2021년 10월).xlsx',
 '대구광역시립두류도서관 장서 대출목록 (2021년 10월).xlsx',
 '경상남도교육청 김해지혜의바다도서관 장서 대출목록 (2021년 10월).xlsx',
 '충청남도교육청 학생교육문화원 장서 대출목록 (2021년 10월).xlsx',
 '가평군 청평도서관 장서 대출목록 (2021년 10월).xlsx',
 '청주흥덕도서관 장서 대출목록 (2021년 10월).xlsx',
 '대구 동구 안심도서관 장서 대출목록 (2021년 10월).xlsx',
 '시흥시 군자도서관 장서 대출목록 (2021년 10월).xlsx',
 '도봉아이나라도서관 장서 대출목록 (2021년 10월).xlsx',
 '경기평택교육도서관 장서 대출목록 (2021년 10월).xlsx']

In [19]:
path + '\\' + tmp[0]

'C:\\Users\\user\\Desktop\\도서관\\스크래핑\\대출목록\\한밭도서관 장서 대출목록 (2021년 10월).xlsx'

In [20]:
# 정제된 데이터를 저장할 월간교통량_평균 파일 생성
writer = pd.ExcelWriter('도서관별 대출건수(2019-2021).xlsx', engine = 'xlsxwriter')

In [22]:
for i in tmp:
    print(i)
    # df = pd.read_excel(path + '\\' + i, engine = 'openpyxl', usecols=[11, 12])

한밭도서관 장서 대출목록 (2021년 10월).xlsx
광주학생교육문화회관 장서 대출목록 (2021년 10월).xlsx
울산도서관 장서 대출목록 (2021년 10월).xlsx
남양주시 오남도서관 장서 대출목록 (2021년 10월).xlsx
고척도서관 장서 대출목록 (2021년 10월).xlsx
남양주시 진접푸른숲도서관 장서 대출목록 (2021년 10월).xlsx
대구광역시립두류도서관 장서 대출목록 (2021년 10월).xlsx
경상남도교육청 김해지혜의바다도서관 장서 대출목록 (2021년 10월).xlsx
충청남도교육청 학생교육문화원 장서 대출목록 (2021년 10월).xlsx
가평군 청평도서관 장서 대출목록 (2021년 10월).xlsx
청주흥덕도서관 장서 대출목록 (2021년 10월).xlsx
대구 동구 안심도서관 장서 대출목록 (2021년 10월).xlsx
시흥시 군자도서관 장서 대출목록 (2021년 10월).xlsx
도봉아이나라도서관 장서 대출목록 (2021년 10월).xlsx
경기평택교육도서관 장서 대출목록 (2021년 10월).xlsx


In [23]:
for i in tmp:
    print(i)
    df = pd.read_excel(path + '\\' + i, engine = 'openpyxl', usecols=[11, 12])
    ddf = df.copy()
    print("read " + i)
    
    ddf = ddf[ddf['등록일자'].str.contains('2019') | ddf['등록일자'].str.contains('2020') | ddf['등록일자'].str.contains('2021')]

    # 특정문자 기준으로 문자열 나누고 새로운 열 추가
    ddf['년'] = ddf.등록일자.str.split('-').str[0]
    ddf['월'] = ddf.등록일자.str.split('-').str[1]
    ddf['일'] = ddf.등록일자.str.split('-').str[2]

    # 월별 대출 건수
    grouped_df = ddf.groupby(['년', '월']).sum()
    grouped_df
    
    grouped_df.to_excel(writer, index = True, sheet_name = i[:i.find('장') - 1])

writer.save()
    

한밭도서관 장서 대출목록 (2021년 10월).xlsx
read 한밭도서관 장서 대출목록 (2021년 10월).xlsx
광주학생교육문화회관 장서 대출목록 (2021년 10월).xlsx
read 광주학생교육문화회관 장서 대출목록 (2021년 10월).xlsx
울산도서관 장서 대출목록 (2021년 10월).xlsx
read 울산도서관 장서 대출목록 (2021년 10월).xlsx
남양주시 오남도서관 장서 대출목록 (2021년 10월).xlsx
read 남양주시 오남도서관 장서 대출목록 (2021년 10월).xlsx
고척도서관 장서 대출목록 (2021년 10월).xlsx
read 고척도서관 장서 대출목록 (2021년 10월).xlsx
남양주시 진접푸른숲도서관 장서 대출목록 (2021년 10월).xlsx
read 남양주시 진접푸른숲도서관 장서 대출목록 (2021년 10월).xlsx
대구광역시립두류도서관 장서 대출목록 (2021년 10월).xlsx
read 대구광역시립두류도서관 장서 대출목록 (2021년 10월).xlsx
경상남도교육청 김해지혜의바다도서관 장서 대출목록 (2021년 10월).xlsx
read 경상남도교육청 김해지혜의바다도서관 장서 대출목록 (2021년 10월).xlsx
충청남도교육청 학생교육문화원 장서 대출목록 (2021년 10월).xlsx
read 충청남도교육청 학생교육문화원 장서 대출목록 (2021년 10월).xlsx
가평군 청평도서관 장서 대출목록 (2021년 10월).xlsx
read 가평군 청평도서관 장서 대출목록 (2021년 10월).xlsx
청주흥덕도서관 장서 대출목록 (2021년 10월).xlsx
read 청주흥덕도서관 장서 대출목록 (2021년 10월).xlsx
대구 동구 안심도서관 장서 대출목록 (2021년 10월).xlsx
read 대구 동구 안심도서관 장서 대출목록 (2021년 10월).xlsx
시흥시 군자도서관 장서 대출목록 (2021년 10월).xlsx
read 시흥시 군자도서관 장서 대출목록 (2021년 10월).xlsx
도봉아

In [30]:
lib_list

'경기평택교육도서관'

# 그래프 그리기

In [36]:
# new_df = pd.DataFrame()

for i in range(15):
    print(lib_list[i])
    df = pd.read_excel('도서관별 대출건수(2019-2021).xlsx', engine = 'openpyxl', sheet_name = lib_list[i])
    new_df = pd.concat([new_df, df])
    
new_df    

한밭도서관
광주학생교육문화회관
울산도서관
남양주시 오남도서관
고척도서관
남양주시 진접푸른숲도서관
대구광역시립두류도서관
경상남도교육청 김해지혜의바다도서관
충청남도교육청 학생교육문화원
가평군 청평도서관
청주흥덕도서관
대구 동구 안심도서관
시흥시 군자도서관
도봉아이나라도서관
경기평택교육도서관


,년,월,대출건수
0,2019.0,1,13759
1,NaN,2,17122
2,NaN,3,7375
3,NaN,4,7008
4,NaN,5,12115
...,...,...,...
28,NaN,6,323
29,NaN,7,223
30,NaN,8,1442
31,NaN,9,97


## API 

In [15]:
lib_code_list = []

In [16]:
lib_code = driver.find_element_by_xpath('//*[@id="sb-site"]/section/div[5]/div/table/tbody/tr[1]/td[2]')
lib_code_list.append(lib_code.text)

In [17]:
lib_code_list

['125003']

In [ ]:
# 도서관 API 

- 도서관 정보나루
- https://www.data4library.kr/apiUtilization

- 도서관별 통합정보 -> 도서관별 장서/대출 데이터 조회

- 도서관정보나루_API_Manual.hwp 를 참고하여 작성

- 도서관명 / 

In [27]:
import urllib.request
import json
import pandas as pd
from datetime import datetime,timedelta
import requests
import xmltodict


key = '9e70f846d07f0a8577bf0eec924305c22cec843db920f6ed5c3c0ca4cc871b03'
startDt = '2019-01-01'
endDt = '2021-10-31'

libCode = lib_code_list[0]

# Call Back URL + 인증키 + 날짜

url = 'http://data4library.kr/api/itemSrch?libCode=' + libCode + '&authKey=' + key + '&startDt=' + startDt + '&endDt=' + endDt + '&pageNo=1&pageSize=1000'

url

'http://data4library.kr/api/itemSrch?libCode=125003&authKey=9e70f846d07f0a8577bf0eec924305c22cec843db920f6ed5c3c0ca4cc871b03&startDt=2019-01-01&endDt=2021-10-31&pageNo=1&pageSize=1000'

In [28]:
req = requests.get(url).content

xmlObject = xmltodict.parse(req)
xmlObject

ExpatError: mismatched tag: line 6, column 2

In [22]:
len(xmlObject['response']['docs']['doc'])

10

In [6]:
# dict_data = xmlObject['response']['libs']['lib'][0]['loanByhours']['result']
dict_data = xmlObject['response']['docs']['doc'][0]

df_conf = pd.DataFrame(dict_data)

In [7]:
df_conf

,bookname,authors,publisher,publication_year,isbn13,set_isbn13,bookImageURL,addition_symbol,vol,class_no,class_nm,callNumbers,reg_date
callNumber,산다는 건 뭘까?,채인선,미세기,2021,9788980715121,None,https://image.aladin.co.kr/product/27858/76/co...,None,None,126.5,"철학 > 인식론, 인과론, 인간학 >","{'separate_shelf_code': '아', 'separate_shelf_n...",2021-10-27
